In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import il_tutorial.cost_graphs as cg
import il_tutorial.util as util
from IPython.display import HTML

In [8]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

<center>
<h2>Imitation learning for structured prediction <br> in natural language processing</h2>
<p style="text-align:center">
<a href="http://andreasvlachos.github.io">Andreas Vlachos</a><br>
a.vlachos@sheffield.ac.uk<br>
<small>Department of Computer Science<br>
University of Sheffield
</small>
</p>
<br>
<p class="fragment" style="text-align:center; font-size: 85%">
Joint work with Gerasimos Lampouras (Sheffield), Sebastian Riedel (UCL),<br> Daniel Beck (Sheffield), Jason Naradowsky (UCL), James Goodman (UCL),<br> Isabelle Augenstein (Sheffield), Stephen Clark (Cambridge),<br> Mark Craven (Wisconsin-Madison)
</p>

</center>

<h3>Structured prediction in Natural Language Processing</h3>

<table  style="border-style: hidden; border-collapse: collapse; padding: 50px">
<thead>
<tr>
<th>I</th> 
<th>studied</th>
<th>in</th>
<th>London</th>
<th>with</th>
<th>Sebastian</th>
<th>Riedel</th>
</tr>
</thead>
<tbody style="font-size:100%">
<tr>
<td><span class="fragment" data-fragment-index="1">PRP</span></td>
<td><span class="fragment" data-fragment-index="1">VBD</span></td>
<td><span class="fragment" data-fragment-index="1">IN</span></td>
<td><span class="fragment" data-fragment-index="1">NNP</span></td>
<td><span class="fragment" data-fragment-index="1">IN</span></td>
<td><span class="fragment" data-fragment-index="1">NNP</span></td>
<td><span class="fragment" data-fragment-index="1">NNP</span></td>
</tr>
<tr>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">B-LOC</span></td>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">B-PER</span></td>
<td><span class="fragment" data-fragment-index="2">I-PER</span></td>
</tr>
</tbody>
</table>

<p>
				<ul>
  			<li class="fragment" data-fragment-index="1">part of speech (PoS) tagging</li>
  			<li class="fragment" data-fragment-index="2">named entity recognition (NER)</li>
				</ul>
			</p>


<p><b>Input:</b> a sentence $\mathbf{x}=[x_1...x_N]$<br> <b>Output:</b> a sequence of labels $\mathbf{y}=[y_{1}\ldots y_{N}] \in {\cal Y}^N$</p>

<h3>More Structured Prediction</h3>
<center>
<img src="images/toBeAnimated/depParse1.png" style="width:90%;">

<img class="fragment" src="images/tikz/semParse.png" style="width:60%;">
</center>

<p>Syntactic parsing, semantic parsing, semantic role labeling, question answering over knowledge bases, etc.</p>
<p><b>Input:</b> a sentence $\mathbf{x}=[x_1...x_N]$<br>
<b>Output:</b> a meaning representation graph $\mathbf{G}=(V,E) \in {\cal G_{\mathbf{x}}}$</p> 

<h3>More Structured Prediction</h3>

<img src="images/nlg.png" style="width:80%;">

<p>Natural language generation (NLG), but also summarization, decoding in machine translation, etc.</p>

<p><b>Input:</b> a meaning representation<br>
<b>Output:</b> $\mathbf{w}=[w_1...w_N], w\in {\cal V}\cup END, w_N=END$</p>  

### This talk: Imitation Learning

<p style="float: left;">We assume gold standard<br> output for <b>supervised</b> training</p> 
<img src="images/tikz/StucturedPredictionDef.png" style="width:35%; float: right;">

<p style="float: left;">But we train a classifier to predict<br> <b>actions</b> constructing the output.<br><br>Actions not in gold;<br> IL is rather <b>semi-supervised</b></p> 
<img src="images/tikz/StucturedPrediction.png" style="width:35%; float: right;">

### Imitation learning in a nutshell


<a href="http://www.cs.cmu.edu/~sross1/publications/Ross-AIStats11-Slides.pdf"><img src="images/imitationFromRoss.png" style="width:75%;"></a>


**Meta-learning**: better model (&asymp;policy) by generating better training data from expert demonstrations. 

### Is it reinforcement learning?

<p style="float: left;"><b>Yes</b> (a kind of): we train a policy to<br>maximize rewards/minimize losses</p> 
<a href="http://www.clipartkid.com/you-never-learn-to-swear-until-you-teach-your-teenager-to-drive-d85J9Y-clipart/"><img src="images/driver.jpg" style="width:25%; float: right;"></a>

<p style="float: left;">But learning is facilitated by an <b>expert</b><br></p>  <a href="https://www.pinterest.com/explore/affordable-driving-school/"><img src="images/driving_mix.jpg" style="width:30%; float: right;"></a>

### Why should I care?

In NLP we train classifiers to imitate experts in many tasks:
- syntactic parsing ([Ballesteros et al., 2016](https://arxiv.org/pdf/1603.03793.pdf))
- cofererence resolution ([Clark and Manning, 2015](http://cs.stanford.edu/people/kevclark/resources/clark-manning-acl15-entity.pdf))
- semantic parsing ([Goodman et al., 2016](http://aclweb.org/anthology/P16-1001))
- natural language generation ([Lampouras and Vlachos, 2016](https://aclweb.org/anthology/C/C16/C16-1105.pdf))

Imitation learning has been used to improve accuracy in all the above with SOTA results!

### Talk structure

- Structured prediction preliminaries
- Imitation learning
  - PoS tagging example
  - relation to other learning paradigms
- Applications
  - Semantic parsing
  - Natural language generation

<center>
<h2>Structured Prediction Preliminaries</h2>
</center>

### Two main paradigms

Joint modeling, a.k.a: 
- global inference
- structured models

Incremental modeling, a.k.a:
- local 
- greedy
- pipeline
- transition-based
- history-based

### Joint modeling

A model (e.g. conditional random fields) that scores complete outputs (e.g. label sequences):

$$\mathbf{\hat y} =\hat y_{1}\ldots \hat y_{N} = \mathop{\arg \max}_{Y \in {\cal Y}^N} f(y_{1}\ldots y_{N}, \mathbf{x})$$

<ul class="fragment">
					<li>exhaustive exploration of the search space</li>
					<li>large/complex search spaces are challenging</li>
					<li>efficient dynamic programming restricts modelling flexibility
						(i.e. Markov assumptions)</li>
				</ul>


### Incremental modeling

A classifier predicting a label at a time given the previous ones:


\begin{align}
\hat y_1 &=\mathop{\arg \max}_{y \in {\cal Y}} f(y, \mathbf{x}),\\
\mathbf{\hat y} = \quad \hat y_2 &=\mathop{\arg \max}_{y \in {\cal Y}} f(y, \mathbf{x}, \hat y_1), \cdots\\
\hat y_N &=\mathop{\arg \max}_{y \in {\cal Y}} f(y, \mathbf{x}, \hat y_{1} \ldots \hat y_{N-1})
\end{align}

<ul class="fragment">
					<li>use our favourite classifier</li>
					<li>no restrictions on features</li>
					<li>prone to error propagation (i.i.d. assumption broken)</li>
					<li>local model not trained wrt the task-level loss</li>
				</ul>


### Imitation learning

Improve incremental modeling to:
- address error-propagation
- train wrt the task-level loss function

**Meta-learning**: use our favourite classifier and features,
but generate better (non-i.i.d.) training data

To apply IL we need:
- transition system (what our classifier can do)
- task loss (what we optimize for)
- expert policy (the teacher to help us)

<h3>Transition system</h3>

<p>The <b>actions</b> $\cal A$ the classifier $f$ can predict and their effect on the <b>state</b> which tracks the prediction: $S_{t+1}=S_1(\alpha_1\ldots\alpha_t)$</p>

<img src="images/tikz/IncrementalStructure.png" style="align:center; width:60%">

<h3>Transition system</h3>

<p style="text-align: left; border:3px; border-radius: 25px; background-color:lightgrey; border-style:solid; border-color:black; padding: 0.3em; font-size: 75%">
\begin{align}
& \textbf{Input:} \; sentence \; \mathbf{x}\\
& state \; S_1=initialize(\mathbf{x}); timestep \; t = 1\\
& \mathbf{while}\; S_t \; \text{not final}\; \mathbf{do}\\
& \quad action \; \alpha_t = \mathop{\arg \max}_{\alpha \in {\cal A}} f(\alpha, \mathbf{x})\\
& \quad S_{t+1}=S_t(\alpha_t); t=t+1\\
\end{align}
</p>

<ul>
<li><b>PoS/NER tagging?</b> <span class="fragment">for each word, left-to-right, predict a PoS/NER tag which is added to the output</span></li>
<li class="fragment"><b>NLG?</b> <span class="fragment">predict a word adding it to the output until the <code>EndOfSentence</code> is predicted</span></li>
</ul>

### Supervising the classifier

What are good actions in incremental structured prediction?

Those that reach $S_{final} = S_1(\alpha_1\ldots\alpha_T)$ with low **task loss**:

$$loss  = L(S_{final}, \mathbf{y}) \geq 0$$

<ul>
<li><b>PoS tagging?</b> <span class="fragment">Hamming loss: number of incorrect tags</span></li>
<li class="fragment"><b>NER?</b> <span class="fragment">number of false positives and false negatives</span></li>
<li class="fragment"><b>NLG?</b> <span class="fragment">BLEU: % of n-grams predicted present in the gold reference(s), i.e. $L=1-BLEU(S_{final}, \mathbf{y})$</span></li>
</ul>

### Action assessment 

<table style="font-size:80%; border-style: hidden; border-collapse: collapse; padding: 50px">
<thead>
<tr>
<th>I</th> 
<th>studied</th>
<th>in</th>
<th>London</th>
<th>with</th>
<th>Sebastian</th>
<th>Riedel</th>
</tr>
</thead>
<tbody>
<tr>
<td>PRP</td>
<td>VBD</td>
<td>IN</td>
<td>NNP</td>
<td>IN</td>
<td>NNP</td>
<td><span class="fragment" data-fragment-index="1">NNP</span></td>
</tr>
<tr>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">B-LOC</span></td>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">B-PER</span></td>
<td><span class="fragment" data-fragment-index="3">I-PER</span></td>
</tr>
</tbody>
</table>

<p>How many incorrect PoS tags due to $\alpha_6$  being NNP? <span class="fragment" data-fragment-index="1"><b>0</b></span>
</p>
<p class="fragment" data-fragment-index="2"> How many $FP+FN$ due to $\alpha_6$ being B-PER? <span class="fragment" data-fragment-index="3"><br><b>Depends!</b> If $\alpha_7$ is</span>
<ul class="fragment" data-fragment-index="3">
<li>I-PER:  $0$ (correct)</li> 
<li>O: $2$ (1FP+1FN)</li>
<li>B-*: $3$ (2FP+1FN)</li>
</ul>
</p>
<p class="fragment" data-fragment-index="3">$FP+FN$ loss is <b>non-decomposable</b> wrt the transition system
</p>

### More action assessment

Can we tell how predicting a word will change our BLEU?
$$
BLEU([\alpha_1...\alpha_T],\mathbf{y}) = \prod_{n=1}^N \frac{\# \text{n-grams} \in ([\alpha_1...\alpha_T] \cap \mathbf{y})}{\# \text{n-grams} \in [\alpha_1..\alpha_T]}
$$
<p class="fragment" data-fragment-index="1"><b>No</b>! (assuming N>1)</p>

**Non-decomposable** losses wrt transition system are common!

Imitation learning trains incremental models for such losses

Affects joint models too: loss does not always decompose over the graphical model ([Tarlow and Zemel, 2012](http://www.cs.toronto.edu/~dtarlow/tarlow_zemel_aistats12.pdf))

### Expert policy

Returns the best action at the current state by looking at the gold standard assuming **future actions are also optimal**:

$$\alpha^{\star}=\pi^{\star}(S_t, \mathbf{y}) = \mathop{\arg \min}_{\alpha \in {\cal A}} L(S_t(\alpha,\pi^{\star}),\mathbf{y})$$

<p style="float: left;">Only available for the training data: an expert<br>demonstrating how to perform the task </p> <a href="http://www.salon.com/2016/10/06/what-makes-a-good-teacher-why-certifications-and-standards-dont-guarantee-quality-educators_partner/"><img src="images/english_teacher.jpg" style="width:25%; float: right;"></a>

### Expert policy

What action should $\pi^{\star}$ return?

<table style="border-style: hidden; border-collapse: collapse; padding: 50px">
<thead>
<tr>
<th>I</th> 
<th>studied</th>
<th>in</th>
<th>London</th>
<th>with</th>
<th>Sebastian</th>
<th>Riedel</th>
</tr>
</thead>
<tbody>
<tr>
<td>O</td>
<td>O</td>
<td>O</td>
<td>B-LOC</td>
<td>O</td>
<td>B-PER</td>
<td><span class="fragment" data-fragment-index="1">I-PER</span></td>
</tr>
<tr class="fragment" data-fragment-index="2">
<td>O</td>
<td>O</td>
<td>O</td>
<td>B-LOC</td>
<td>O</td>
<td>O</td>
<td><span class="fragment" data-fragment-index="3">O</span></td>
</tr>
</tbody>
</table> 

Takes previous actions into account (**dynamic** vs **static**)

Finding the optimal action can be expensive but we can learn with **sub-optimal** experts.

<center>
<h2>Imitation learning</h2>
</center>

### Imitation learning for part-of-speech tagging

<table style="border-style: hidden; border-collapse: collapse; padding: 50px">
<thead>
<tr>
<th>I</th>
<th>can</th>
<th>fly</th>
</tr>
</thead>
<tbody>
<tr>
<td><span>Pronoun</span></td>
<td><span>Modal</span></td>
<td><span>Verb</span></td>
</tr>
</tbody>
</table>

 **Task loss**: <span class="fragment">Hamming loss: number of incorrectly predicted tags</span>

**Transition system**: <span class="fragment">Tag each token left-to-right</span>

**Expert policy**: <span class="fragment">Return the next tag from the gold standard</span>

<h3>Gold standard in search space</h3>

In [9]:
paths = [[],[(0,4),(1,3)],[(0,4),(1,3),(2,2)],[(0,4),(1,3),(2,2),(3,1)]]
rows = ['Noun', 'Verb', 'Modal', 'Pronoun','NULL']
columns = ['NULL','I', 'can', 'fly']
cbs = []
for path in paths:
    cbs.append(cg.draw_cost_breakdown(rows, columns, path))
util.Carousel(cbs)

<p>
<ul>
<li>Three actions to complete the output</li>
<li>Expert policy replicates the gold standard</li>
</ul>
</p>

<h3>Training a classifier<span class="fragment" data-fragment-index="1"> with structure features </span></h3>

In [10]:
gold_path = [(0,4),(1,3),(2,2),(3,1)]
cb_gold = cg.draw_cost_breakdown(rows, columns, gold_path)
cb_gold

<table style="font-size:100%; border-style:hidden; border-collapse:collapse; padding:50px; float:left;">
<thead>
<tr>
<th>timestep</th>
<th>label ($\alpha_t$)</th>
<th>features ($\phi(S_{t-1},\mathbf{x})$)</th>
</tr>
</thead>
<tbody>
<tr>
<td> $t=1$ </td>
<td><b>Pronoun</b></td>
<td>token=I, ...<span class="fragment" data-fragment-index="1">, prev=<b>NULL</b></span></td>
</tr>
<tr>
<td> $t=2$ </td>
<td><b>Modal</b></td>
<td>token=can, ...<span class="fragment" data-fragment-index="1">, prev=<b>Pronoun</b></span></td>
</tr>
<tr>
<td> $t=3$ </td>
<td><b>Verb</b></td>
<td>token=fly, ...<span class="fragment" data-fragment-index="1">, prev=<b>Modal</b></span></td>
</tr>
</tbody>
</table>

### Algorithm

<p style="border:3px; border-radius: 25px; background-color:lightgrey; border-style:solid; border-color:black; padding: 0.3em; font-size: 80%">
\begin{align}
& \textbf{Input:} \; D_{train} = \{(\mathbf{x}^1,\mathbf{y}^1)...(\mathbf{x}^M,\mathbf{y}^M)\}, \; \text{expert}\; \pi^{\star}, \; \text{classifier} \; H\\
& \text{set training examples}\; \cal E = \emptyset\\
& \mathbf{for} \; (\mathbf{x},\mathbf{y}) \in D_{train} \; \mathbf{do}\\
& \quad \text{generate expert trajectory} \; \alpha_1^{\star}\dots \alpha_T^{\star}  = \pi^{\star}(\mathbf{x},\mathbf{y})\\
& \quad \mathbf{for} \; \alpha^{\star}_t \in \alpha_1^{\star}\dots \alpha_T^{\star} \; \mathbf{do}\\
& \quad \quad \text{extract features}\; \mathit{feat}=\phi(\mathbf{x},S_{t-1}) \\
& \quad \quad \cal E = \cal E \cup (\mathit{feat},\alpha^{\star}_t)\\
& \text{learn} \; H\; \text{from}\; \cal E\\
\end{align}
</p>

### Exposure bias

In [11]:
wrong_path = [(0,4),(1,3),(2,1)]
cb_wrong = cg.draw_cost_breakdown(rows, columns, wrong_path)
util.Carousel([cb_gold, cb_wrong])

<p style="float: left; font-size: 80%">We had seen: &nbsp;&nbsp; 
<table style="float: left; border-style: hidden; border-collapse: collapse; font-size: 80%">
<thead>
<tr>
<th>timestep</th>
<th>label</th>
<th>features</th>
</tr>
</thead>
<tbody>
<tr>
<td>t=3</td>
<td><b>Verb</b></td>
<td>token=fly,..., prev=<b>Modal</b></td>
</tr>
</tbody>
</table>
</p>

<p style="float: left; font-size: 80%">but not: &nbsp;&nbsp;
<table style="float: left; border-style: hidden; border-collapse: collapse; font-size: 80%">
<thead>
<tr>
<th>timestep</th>
<th>label</th>
<th>features</th>
</tr>
</thead>
<tbody>
<tr>
<td>t=3</td>
<td><b>Verb</b></td>
<td>token=fly,..., <span style="color:red">prev=<b>Verb</b></span></td>
</tr>
</tbody>
</table></p>

### Addressing exposure with Rollins

<p style="float: left;">Allow the classifier to guide the learning<br></p>  <a href="https://www.pinterest.com/explore/affordable-driving-school/"><img src="images/driving_mix.jpg" style="width:35%; float: right;"></a>

Define a **rollin** policy that sometimes uses the expert $\pi^{\star}$ and other times the classifier $H$:

$$\pi^{in} = \beta\pi^{\star} + (1-\beta)H$$

### DAgger algorithm

<p style="border:3px; border-radius: 25px; background-color:lightgrey; border-style:solid; border-color:black; padding: 0.3em; font-size: 80%">
\begin{align}
& \textbf{Input:} \; D_{train} = \{(\mathbf{x}^1,\mathbf{y}^1)...(\mathbf{x}^M,\mathbf{y}^M)\}, \; \text{expert}\; \pi^{\star}, \; \text{classifier} \; H\\
& \text{set training examples}\; \cal E = \emptyset ,\; \color{red}{\pi^{\star}\; \mathrm{probability}\; \beta=1}\\
& \mathbf{while}\; \text{termination condition not reached}\; \mathbf{do}\\
& \quad \color{red}{\text{set rollin policy} \; \pi^{in} = \beta\pi^{\star} + (1-\beta)H}\\
& \quad \mathbf{for} \; (\mathbf{x},\mathbf{y}) \in D_{train} \; \mathbf{do}\\
& \quad \quad \color{red}{\text{generate trajectory} \; \hat \alpha_1\dots\hat \alpha_T  = \pi^{in}(\mathbf{x},\mathbf{y})}\\
& \quad \quad \mathbf{for} \; \hat \alpha_t \in \hat \alpha_1\dots\hat \alpha_T \; \mathbf{do}\\
& \quad \quad \quad \color{red}{\text{ask expert for best action}\; \alpha^{\star} = \pi^{\star}(\mathbf{x}, \mathbf{y},S_{t-1})} \\
& \quad \quad \quad \text{extract features} \; \mathit{feat}=\phi(\mathbf{x},S_{t-1}) \\
& \quad \quad \quad \cal E = \cal E \cup (\mathit{feat},\alpha^{\star})\\
& \quad \text{learn}\; H \; \text{from}\; \cal E\\
& \quad \color{red}{\text{decrease} \; \beta}\\
\end{align}
</p>

### DAgger algorithm ([Ross et al., 2011](http://www.cs.cmu.edu/~sross1/publications/Ross-AIStats11-NoRegret.pdf))

- first iteration is standard classification training
- subsequent ones generate training examples from non-expert trajectories
- task loss is implicitly considered via the expert
- DAgger: the Datasets in each iteration are Aggregated

**rollins** help recover from previous mistakes. How do we learn the future impact of a mistake?

**rollout**: try each action available and see what happens when future actions are taken by mixing the classifier and the expert

### Training labels as costs

In [12]:
cb_gold

<table style="float: left; border-style: hidden; border-collapse: collapse; float:left;">
<thead>
<tr>
<th>timestep</th>
<th><b>Pronoun</b></th>
<th><b>Modal</b></th>
<th><b>Verb</b></th>
<th><b>Noun</b></th>
<th>features</th>
</tr>
</thead>
<tbody>
<tr>
<td>t=1</td>
<td>0</td>
<td>1</td>
<td>1</td>
<td>1</td>
<td>token=I, prev=<b>NULL</b>...</td>
</tr>
<tr>
<td>t=2</td>
<td>1</td>
<td>0</td>
<td>1</td>
<td>1</td>
<td>token=can, prev=<b>Pronoun</b>...</td>
</tr>
<tr>
<td>t=3</td>
<td>1</td>
<td>1</td>
<td>0</td>
<td>1</td>
<td>token=fly, prev=<b>Modal</b>...</td>
</tr>
</tbody>
</table>

<h3>Cost break down</h3>

In [13]:
p = gold_path.copy()
cost = 1
cb_costs = []
cb_costs.append(cg.draw_cost_breakdown(rows, columns, [(0,4),(1,3)], roll_in_cell=p[1]))
cb_costs.append(cg.draw_cost_breakdown(rows, columns, [(0,4),(1,3),(2,0)], roll_in_cell=p[1], explore_cell=(2,0)))
cb_costs.append(cg.draw_cost_breakdown(rows, columns, [(0,4),(1,3),(2,0),(3,1)], roll_in_cell=p[1], explore_cell=(2,0),roll_out_cell=(3,0)))
cb_costs.append(cg.draw_cost_breakdown(rows, columns, [(0,4),(1,3),(2,0),(3,1)], cost, p[3], roll_in_cell=p[1], explore_cell=(2,0),roll_out_cell=(3,0)))
for i in range(1,4):
    p = gold_path.copy()
    p[2] = (gold_path[2][0],i)
    if p == gold_path:
        cost = 0
    else:
        cost = 1
    cb_costs.append(cg.draw_cost_breakdown(rows, columns, p, cost, p[3], roll_in_cell=p[1],roll_out_cell=(3,0), explore_cell=p[2]))
util.Carousel(cb_costs)

<table style="border-style: hidden; border-collapse: collapse; font-size: 90%">
<thead>
<tr>
<th>step</th>
<th><b>Pronoun</b></th>
<th><b>Modal</b></th>
<th><b>Verb</b></th>
<th><b>Noun</b></th>
<th>features</th>
</tr>
</thead>
<tbody>
<tr>
<td>t=2</td>
<td>1</td>
<td>0</td>
<td>1</td>
<td>1</td>
<td>token=can, prev=<b>Pronoun</b>...</td>
</tr>
</tbody>
</table>
<ul style="font-size: 90%">
<li><b>rollin</b> to a point in the sentence</li>
<li><b>explore</b> each action: <b>rollout</b> and cost with task loss</li>
</ul>

### Mixed rollouts

<p>
$$\pi^{out} = \beta\pi^{\star} + (1-\beta)H$$
</p>

In [14]:
cb_mix_costs = []
cb_mix_costs.append(cg.draw_cost_breakdown(rows, columns, [(0,4),(1,3)], roll_in_cell=p[1]))
cb_mix_costs.append(cg.draw_cost_breakdown(rows, columns, [(0,4),(1,3),(2,0)], roll_in_cell=p[1], explore_cell=(2,0)))
for i in range(4):
    p = gold_path.copy()
    p[2] = (gold_path[2][0],i)
    if p == gold_path:
        cost = 0
    elif i==1:
        cost =2
        p[3] = (3,0)
    else:
        cost = 1
    cb_mix_costs.append(cg.draw_cost_breakdown(rows, columns, p, cost, p[3], roll_in_cell=p[1],roll_out_cell=(3,0), explore_cell=p[2]))
util.Carousel(cb_mix_costs)

<table style="border-style: hidden; border-collapse: collapse; padding: 50px;">
<thead>
<tr>
<th><b>Pronoun</b></th>
<th><b>Modal</b></th>
<th><b>Verb</b></th>
<th><b>Noun</b></th>
<th>features</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>0</td>
<td>2</td>
<td>1</td>
<td>token=can, prev=<b>Pronoun</b>...</td>
</tr>
</tbody>
</table>

### DAgger with roll-outs

<p style="border:3px; border-radius: 25px; background-color:lightgrey; border-style:solid; border-color:black; padding: 0.3em; font-size: 70%">
\begin{align}
& \textbf{Input:} \; D_{train} = \{(\mathbf{x}^1,\mathbf{y}^1)...(\mathbf{x}^M,\mathbf{y}^M)\}, \; \text{expert}\; \pi^{\star}, \; \text{classifier} \; H, \; \text{loss} \; L\\
& \text{set training examples}\; \cal E = \emptyset, \; \pi^{\star}\; \mathrm{probability}\; \beta=1\\
& \mathbf{while}\; \text{termination condition not reached}\; \mathbf{do}\\
& \quad \color{red}{\text{set rollin/out policy} \; \pi^{in/out} = \beta\pi^{\star} + (1-\beta)H}\\
& \quad \mathbf{for} \; (\mathbf{x},\mathbf{y}) \in D_{train} \; \mathbf{do}\\
& \quad \quad \text{rollin to predict} \; \hat \alpha_1\dots\hat \alpha_T  = \pi^{in/out}(\mathbf{x},\mathbf{y})\\
& \quad \quad \mathbf{for} \; \hat \alpha_t \in \hat \alpha_1\dots\hat \alpha_T \; \mathbf{do}\\
& \quad \quad \quad \mathbf{for} \; \alpha \in {\cal A} \; \mathbf{do}\\
& \quad \quad \quad \quad \color{red}{\text{rollout} \; S_{final} = \pi^{in/out}(S_{t-1}, \alpha, \mathbf{x})}\\
& \quad \quad \quad \quad \color{red}{\text{cost}\; c_{\alpha}=L(S_{final}, \mathbf{y})}\\
& \quad \quad \quad \text{extract features}\; \mathit{feat}=\phi(\mathbf{x}, S_{t-1}) \\
& \quad \quad \quad \cal E = \cal E \cup (\mathit{feat},\mathbf{c})\\
& \quad \text{learn} \;H \; \text{from}\; \cal E\\
& \quad \text{decrease} \; \beta\\
\end{align}
</p>

### Roll-outs

- non-decomposable loss only used on complete outputs
- first proposed in SEARN ([Daumé III et al., 2009](http://hunch.net/~jl/projects/reductions/searn/searn.pdf))
- used to hybridise DAgger by [Vlachos and Clark (2014)](http://www.aclweb.org/anthology/Q14-1042), referred to later as V-DAgger ([Goodman et al. 2016](http://aclweb.org/anthology/P16-1001))
- also proposed as look-aheads ([Tsuruoka et al. 2011](http://www.anthology.aclweb.org/W/W11/W11-0328.pdf))
- expensive with high variance when long sequences with many actions are possible 
- approximated ([Daumé III et al., 2009](http://hunch.net/~jl/projects/reductions/searn/searn.pdf)), focused ([Vlachos and Craven, 2011](http://www.aclweb.org/anthology/W/W11/W11-0307.pdf)) or used selectively ([Goodman et al. 2016](http://aclweb.org/anthology/P16-1001))

### Locally Optimal Learning to Search (LOLS, [Chang et al., 2015](https://arxiv.org/pdf/1502.02206.pdf))

<img src="images/lols.png" style="width:60%;">
- rollin always with the classifier
- each rollout uses only the expert or the classifier 

### Cost-sensitive classification

Each action has a different cost:

<table style="border-style: hidden; border-collapse: collapse; padding: 50px">
<thead>
<tr>
<th>O</th> 
<th>B-PER</th>
<th>I-PER</th>
<th>B-LOC</th>
<th>I-LOC</th>
<th>B-ORG</th>
<th>I-ORG</th>
</tr>
</thead>
<tbody>
<tr>
<td>2</td>
<td>3</td>
<td>0</td>
<td>3</td>
<td>3</td>
<td>3</td>
<td>3</td>
</tr>
</tbody>
</table>

Learning classifier with costs:
- sample instances according to their cost to train any classifier ([Abe et al., 2004](http://www.hunch.net/~jl/projects/reductions/mc2/p542-Abe.pdf))
- in error-driven learning adjust the updates in proportion to the error cost ([Crammer et al., 2006](http://jmlr.csail.mit.edu/papers/volume7/crammer06a/crammer06a.pdf))

<h3>Generic imitation learning</h3>

<p style="border:3px; border-radius: 25px; background-color:lightgrey; border-style:solid; border-color:black; padding: 0.3em; font-size: 75%">
\begin{align}
& \textbf{Input:} \; D_{train} = \{(\mathbf{x}^1,\mathbf{y}^1)...(\mathbf{x}^M,\mathbf{y}^M)\}, \; \text{expert}\; \pi^{\star}, \text{classifier} \; H, \; \text{loss} \; L\\
& \text{set training examples}\; \cal E = \emptyset\\
& \mathbf{while}\; \text{termination condition not reached}\; \mathbf{do}\\
& \quad \color{red}{\text{set rollin policy} \; \pi^{in} = mix(H,\pi^{\star})}\\
& \quad \color{red}{\text{set rollout policy} \; \pi^{out} = mix(H,\pi^{\star})}\\
& \quad \mathbf{for} \; (\mathbf{x},\mathbf{y}) \in D_{train} \; \mathbf{do}\\
& \quad \quad \color{red}{\text{rollin to predict} \; \hat \alpha_1\dots\hat \alpha_T  = \pi^{in}(\mathbf{x},\mathbf{y})}\\
& \quad \quad \mathbf{for} \; \hat \alpha_t \in \hat \alpha_1\dots\hat \alpha_T \; \mathbf{do}\\
& \quad \quad \quad \color{red}{\text{rollout to obtain costs}\; c \; \text{for all possible actions using}\; L}\\
& \quad \quad \quad \text{extract features}\; \mathit{feat}=\phi(\mathbf{x},S_{t-1}) \\
& \quad \quad \quad \cal E = \cal E \cup (\mathit{feat},c)\\
& \quad \text{learn}\; H \; \text{from}\; \cal E\\
\end{align}
</p>

### Overview

<table style="border-style: hidden">
<thead>
<tr>
<th style="padding: 10px;">Method</th>
<th style="padding: 10px;">rollin</th>
<th style="padding: 10px;">rollout</th>
<th style="padding: 10px;">loss</th>
<th style="padding: 10px;">expert decay</th>
<th style="padding: 10px;">training data</th>
</tr>
</thead>
<tbody>
<tr>
<td style="padding: 10px;">classification</td>
<td style="padding: 10px;">expert</td>
<td style="padding: 10px;">N/A</td>
<td style="padding: 10px;">0/1</td>
<td style="padding: 10px;">N/A</td>
<td style="padding: 10px;">single iteration</td>
</tr>
<tr>
<td style="padding: 10px;">SEARN</td>
<td style="padding: 10px;">mix</td>
<td style="padding: 10px;">mix</td>
<td style="padding: 10px;">task</td>
<td style="padding: 10px;">exponential</td>
<td style="padding: 10px;">weighted averaged across iterations</td>
</tr>
<tr>
<td style="padding: 10px;">DAgger</td>
<td style="padding: 10px;">mix</td>
<td style="padding: 10px;">N/A</td>
<td style="padding: 10px;">0/1</td>
<td style="padding: 10px;">decrease</td>
<td style="padding: 10px;">all iterations</td>
</tr>
<tr>
<td style="padding: 10px;">V-DAgger</td>
<td style="padding: 10px;">mix</td>
<td style="padding: 10px;">mix</td>
<td style="padding: 10px;">task</td>
<td style="padding: 10px;">exponential</td>
<td style="padding: 10px;">all iterations</td>
</tr>
<tr>
<td style="padding: 10px;">LOLS</td>
<td style="padding: 10px;">classifier</td>
<td style="padding: 10px;">action-level mix</td>
<td style="padding: 10px;">task</td>
<td style="padding: 10px;">no decay</td>
<td style="padding: 10px;">averaged across iterations</td>
</tr>
</tbody>
</table>

### Summary so far

- basic intuition behind IL
- rollin and the DAgger algorithm 
- rollouts, V-DAgger and LOLS
- generic imitation learning recipe

<center>
<h2>Applications</h2>
</center>

<h3>Abstract meaning representation (AMR) parsing</h3>
<img src="images/amr.png" style="width:70%; background:white;" />
<p>
<ul>
<li>Designed for semantics-based MT
(<a href="http://amr.isi.edu/a.pdf)">Banarescu et al. 2013</a>)</li>
<li>Many applications: summarization, generation, etc.</li>
<li>See recent tutorial by <a href="http://naacl.org/naacl-hlt-2015/tutorial-amr-semantics.html">Schneider et al. (2015)</a></li>
</ul>
</p>

<h3>Dependency parse to AMR (Wang et al., 2015)</h3>
<img src="images/dep2amr.png" style="width:90%; background:white;" />

<ul>
<li>Start from the dependency parse instead of the sentence</li>
<li>Convert it to AMR by taking a sequence of actions</li>
</ul>

### Similar to transition-based dependency parsing
<center>
<img src="images/stateTransitExpert.png">
</center>

but harder:
- long action sequences (50-200), theoretically unbounded
- many possible actions (~10^4) (each concept is an action)

### Imitation learning for AMR parsing ([Goodman et al.](http://aclweb.org/anthology/P16-1001))

Improved transition-based AMR pasing with
- DAgger (rollins)
- V-DAgger (rollouts)

Two insights:
- targetted exploration
- noise reduction

<h3>Generic imitation learning (repeated)</h3>

<p style="border:3px; border-radius: 25px; background-color:lightgrey; border-style:solid; border-color:black; padding: 0.3em; font-size: 75%">
\begin{align}
& \textbf{Input:} \; D_{train} = \{(\mathbf{x}^1,\mathbf{y}^1)...(\mathbf{x}^M,\mathbf{y}^M)\}, \; \text{expert}\; \pi^{\star}, \text{classifier} \; H, \; \text{loss} \; L\\
& \text{set training examples}\; \cal E = \emptyset\\
& \mathbf{while}\; \text{termination condition not reached}\; \mathbf{do}\\
& \quad \color{red}{\text{set rollin policy} \; \pi^{in} = mix(H,\pi^{\star})}\\
& \quad \color{red}{\text{set rollout policy} \; \pi^{out} = mix(H,\pi^{\star})}\\
& \quad \mathbf{for} \; (\mathbf{x},\mathbf{y}) \in D_{train} \; \mathbf{do}\\
& \quad \quad \color{red}{\text{rollin to predict} \; \hat \alpha_1\dots\hat \alpha_T  = \pi^{in}(\mathbf{x},\mathbf{y})}\\
& \quad \quad \mathbf{for} \; \hat \alpha_t \in \hat \alpha_1\dots\hat \alpha_T \; \mathbf{do}\\
& \quad \quad \quad \color{red}{\text{rollout to obtain costs}\; c \; \text{for all possible actions using}\; L}\\
& \quad \quad \quad \text{extract features}\; \mathit{feat}=\phi(\mathbf{x},S_{t-1}) \\
& \quad \quad \quad \cal E = \cal E \cup (\mathit{feat},c)\\
& \quad \text{learn}\; H \; \text{from}\; \cal E\\
\end{align}
</p>

<h3>Noise reduction</h3>

<p>$\phi(\mathbf{x},\hat y_1\dots \hat y_{n-1})$ describes input and
past actions with discrete features.</p>

<p>Long action histories are difficult to capture:
<ul>
<li>simple features (e.g. previous action) are not discriminative</li>
<li>complex features are sparse and difficult to learn weights</li>
</ul>
</p>

<p class="fragment" data-fragment-index="1">Badly described instances act as noise during training.</p>

<p class="fragment" data-fragment-index="1">
Used the $\alpha$-bound (<a href="http://www.jmlr.org/papers/volume8/khardon07a/khardon07a.pdf">Khardon and Wachman, 2007</a>):
</br>in error-driven training if an instance is misclassified $\alpha$ times, remove it.
</p>

### $\alpha$-bound results ###
<br>
<center>
<img src="images/aboundResults.png" width="60%">
</center>

### Targeted exploration

Rolling out all possible actions can be useful but impractical:
- long action sequences (50-200)
- many possible actions (~10^4)

Rollout (and evaluate the loss) only of:
- the action returned by the expert
- the actions scored by the classifier within a threshold $\tau$ from the highest scoring action

<h3>Targeted exploration</h3>

<img style="float: left;" width="45%" src="images/ReducedExplorationIter.png" style="padding: 0.3em; background:white;">
<img width="47%" src="images/ReducedExploration.png" style="padding: 0.3em; background:white;">
<p>
<ul>
<li>3 iterations of standard V-DAgger (9.6K mins): 65.2</li>
<li>narrow exploration has quick benefits but not optimal</li>
</ul></p>

### Comparison between IL approaches ###

<center>
<img src="images/amrResults_otherIL.png">
</center>

### Comparison against state of the art ###

<center>
<img src="images/semParseRes.png">
</center>

<h3>Natural Language Generation (NLG)</h3>
<img src="images/nlg.png" style="width:80%; background:white;" />
<p>
<ul>
<li>The reverse of semantic parsing</li>
<li>Unlike machine translation, labeled data is rather limited</li>
</ul>
</p>

<h3>Learning NLG</h3>
<img src="images/ContentAndWord.png" style="width:100%; background:white;" />
<p>
<ul>
<li>Two-stage process: content and word prediction</li>
<li>Unaligned training data underspecifies expert policy</li>
</ul>
</p>

### Imitation learning for NLG ([Lampouras and Vlachos 2016](https://aclweb.org/anthology/C/C16/C16-1105.pdf))

Combined elements of [SEARN](http://www.umiacs.umd.edu/~hal/docs/daume09searn.pdf) and [LOLS](https://arxiv.org/abs/1502.02206):
- **rollin** using the classifier (LOLS)
- at each timestep **rollout** with expert or classifier exclusively (LOLS) with decaying probability for the former (SEARN)

Improved performance further by **sequence correction**:
- avoiding generating training examples from very bad rollins
- correct them instead and rollin again

### NLG dev results
<br>
<img src="images/NLG_policyResults.jpg" style="width:70%;">

### Comparison against state of the art
<br>
<img src="images/NLGResults.png">

<center>
<h2>Related work</h2>
</center>

### Reinforcement learning

<p style="float: left;">
<ul style="float: left;">
<li>states defined via features</li>
<li>the agent is a classifier</li>
<li>rewards?</li>
</ul>
</p>

<a href="https://webdocs.cs.ualberta.ca/~sutton/book/ebook/node28.html"><img src="images/RL_sutton.png" style="width:40%; float: right;"></a>

**Inverse reinforcement learning**

- we have the expert policy (inferred from the gold standard in the training data)
- we infer the per-action reward function (rollin/out)				


LOLS with classifier only rollouts is RL ([Chang et al., 2015](https://arxiv.org/pdf/1502.02206.pdf))

<h3> What about Recurrent Neural Networks?</h3>

<img src="images/rnn.png" width="60%"  style="background:none;" />

<p>They face similar problems:
				<ul>
				<li>trained at the word rather than sentence level</li>
				<li>assume previous predictions are correct</li>
			</ul>
			</p>

### Imitation learning and RNNs

<img src="images/mixer.png" width="70%"  style="background:none;" />


- DAgger mixed rollins, similar to scheduled sampling ([Bengio et al., 2015](http://arxiv.org/abs/1506.03099))
- MIXER  (<a href="https://arxiv.org/abs/1511.06732">Ranzato et al., 2016</a>): Mix REINFORCE-ment learning with imitation: we have the expert policy!
- no rollouts, learn a  regressor to estimate action costs
- end-to-end back propagation through the sequence

### Actor-critic

![](images/actorcritic.png)

[Bahdanau et al. (2017)](https://arxiv.org/pdf/1607.07086.pdf):
- actor: the RNN we are learning to use during testing
- critic: another RNN that is trained to predict the value of the actions of the critic

### Summary

Imitation learning:
- better training data for incremental predictors via appropriate training data generation
- addresses error propagation and takes into account the  non-decomposable task loss
- many successful applications
- connections with RL and RNNs

### Some more links

Based on the <a href="http://sheffieldnlp.github.io/ImitationLearningTutorialEACL2017/">EACL2017 tutorial</a> with <a href="http://glampouras.github.io">Gerasimos Lampouras</a> and 
<a href="http://www.riedelcastro.org/">Sebastian Riedel</a>

Code:
<ul>
<li>Vowpal Wabbit Credit assignment compiler ([in Python](http://hunch.net/~vw))</li>
<li>V-DAgger ([in Scala](https://github.com/hopshackle/dagger-AMR)) ([in Python](https://github.com/sheffieldnlp/APEimitaion))</li> 
<li>LOLS ([in Java](https://github.com/glampouras/JLOLS_NLG))
</li> 
<li>MIXER ([in Lua](https://research.fb.com/downloads/mixer/))</li>
</ul>
</p>